# Save and retrieve jobs

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum&trade; so your next session can continue where you left off.

## Retrieve jobs from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs through the [Jobs dashboard](https://quantum.ibm.com/jobs).

To find a job programatically, use the [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) method. By default, this returns the most recent jobs, but you can also filter jobs by backend name, creation date, and more. The following cell finds any jobs submitted in the last three months. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
jobs_in_last_three_months[:3]  # show first three jobs

[<RuntimeJob('cs2dqrz4amg00087hrzg', 'sampler')>,
 <RuntimeJob('cs2bca9kfpw00080febg', 'estimator')>,
 <RuntimeJob('cs2bc5r965y0008531v0', 'estimator')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#job) method to retrieve it.

In [2]:
# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
successful_job = next(j for j in service.jobs() if j.status().name == "DONE")
job_id = successful_job.job_id()
print(job_id)

cs2dqrz4amg00087hrzg


In [3]:
retrieved_job = service.job(job_id)
retrieved_job.result()

PrimitiveResult([PubResult(data=DataBin<>(mid=BitArray(<shape=(), num_shots=4096, num_bits=2>), final=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with Qiskit IBM Runtime's encoders.

In [4]:
import json
from qiskit_ibm_runtime import RuntimeEncoder
with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [5]:
from qiskit_ibm_runtime import RuntimeDecoder
with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result

PrimitiveResult([PubResult(data=DataBin<>(mid=BitArray(<shape=(), num_shots=4096, num_bits=2>), final=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'circuit_metadata': {}})], metadata={'version': 2})

In [6]:
# Cleanup the file we created (this cell should be hidden from the user)
import pathlib
pathlib.Path('result.json').unlink()